In [18]:
from numpy import zeros
from numpy import ones
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
#from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from matplotlib import pyplot as plt
from tensorflow.keras.utils import plot_model
from keras.layers import Input

In [2]:
def define_discriminator(image_shape):
    init=RandomNormal(stddev=0.02)  #weight initialization
    in_src_image=Input(shape=image_shape)
    in_target_image=Input(shape=image_shape)
    merged=Concatenate()([in_src_image,in_target_image])

    d=Conv2D(64,(4,4), strides=(2,2), padding="same", kernel_initializer=init)(merged)
    d=LeakyReLU(alpha=0.2)(d)

    d=Conv2D(128,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(d)
    d=BatchNormalization()(d)
    d=LeakyReLU(alpha=0.2)(d)

    d=Conv2D(256,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(d)
    d=BatchNormalization()(d)
    d=LeakyReLU(alpha=0.2)(d)

    d=Conv2D(512,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(d)
    d=BatchNormalization()(d)
    d=LeakyReLU(alpha=0.2)(d)

    d=Conv2D(512,(4,4),padding="same",kernel_initializer=init)(d) #stride =(1,1)
    d=BatchNormalization()(d)
    d=LeakyReLU(alpha=0.2)(d)

    #patch out
    d=Conv2D(1,(4,4),padding='same',kernel_initializer=init)(d)
    patch_out=Activation('sigmoid')(d)

    #define model
    model=Model([in_src_image,in_target_image],patch_out)

    #Compile model
    opt=Adam(learning_rate=0.0002,beta_1=0.5)
    model.compile(loss='binary_crossentropy',optimizer=opt,loss_weights=[0.5])
    return model
    

In [3]:
test_discr=define_discriminator((64,64,3))
print(test_discr.summary())

C:\Users\IITG\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 64, 64, 6)         │               0 │ input_layer[0][0],         │
│                               │                           │                 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 32, 32, 64)        │           6,208 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu (LeakyReLU)       │ (None, 32, 32, 64)        │               0 │ conv2d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 16, 16, 128)       │         131,200 │ leaky_re_lu[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 16, 16, 128)       │             512 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU)     │ (None, 16, 16, 128)       │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 8, 8, 256)         │         524,544 │ leaky_re_lu_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 8, 8, 256)         │           1,024 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_2 (LeakyReLU)     │ (None, 8, 8, 256)         │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 4, 4, 512)         │       2,097,664 │ leaky_re_lu_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 4, 4, 512)         │           2,048 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_3 (LeakyReLU)     │ (None, 4, 4, 512)         │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 4, 4, 512)         │       4,194,816 │ leaky_re_lu_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 6,968,257 (26.58 MB)

 Trainable params: 6,965,441 (26.57 MB)

 Non-trainable params: 2,816 (11.00 KB)

None


In [5]:
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    #weight initialization
    init=RandomNormal(stddev=0.02)
    #add downsampling layer
    g=Conv2D(n_filters,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(layer_in)
    #conditionally and batch normalization
    if batchnorm:
        g=BatchNormalization()(g,training=True)
    g=LeakyReLU(alpha=0.2)(g)
    return g
    

In [6]:
def decoder_block(layer_in,skip_in,n_filters,dropout=True):
    #weight initialization
    init=RandomNormal(stddev=0.02)
    g=Conv2DTranspose(n_filters,(4,4),strides=(2,2),padding='same',kernel_initializer=init)(layer_in)
    g=BatchNormalization()(g,training=True)
    if dropout:
        g=Dropout(0.5)(g,training=True)
    #merge with skip connection
    g=Concatenate()([g,skip_in])
    g=Activation('relu')(g)
    return g

In [9]:
def define_generator(image_shape=(256,256,3)):
    init=RandomNormal(stddev=0.02)
    in_image=Input(shape=image_shape)

    #encoder model
    e1=define_encoder_block(in_image,64,batchnorm=False)
    e2=define_encoder_block(e1,128)
    e3=define_encoder_block(e2,256)
    e4=define_encoder_block(e3,512)
    e5=define_encoder_block(e4,512)
    e6=define_encoder_block(e5,512)
    e7=define_encoder_block(e6,512)

    #bottleneck
    b=Conv2D(512,(4,4),strides=(2,2),padding='same',kernel_initializer=init)(e7)
    b=Activation('relu')(b)

    #decoder model
    d1=decoder_block(b,e7,512)
    d2=decoder_block(d1,e6,512)
    d3=decoder_block(d2,e5,512)
    d4=decoder_block(d3,e4,512,dropout=False)
    d5=decoder_block(d4,e3,256,dropout=False)
    d6=decoder_block(d5,e2,128,dropout=False)
    d7=decoder_block(d6,e1,64,dropout=False)

    #output
    g=Conv2DTranspose(image_shape[2],(4,4),strides=(2,2),padding='same',kernel_initializer=init)(d7)
    out_image=Activation('tanh')(g)
    model=Model(in_image,out_image)
    return model

In [20]:
gen_model=define_generator()
#plot_model(gen_model,to_file='gen_model.png',show_shapes=True)
print(gen_model.summary())

C:\Users\IITG\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)    │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_54 (Conv2D)            │ (None, 128, 128, 64)      │           3,136 │ input_layer_8[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_47 (LeakyReLU)    │ (None, 128, 128, 64)      │               0 │ conv2d_54[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_55 (Conv2D)            │ (None, 64, 64, 128)       │         131,200 │ leaky_re_lu_47[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_82        │ (None, 64, 64, 128)       │             512 │ conv2d_55[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_48 (LeakyReLU)    │ (None, 64, 64, 128)       │               0 │ batch_normalization_82[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_56 (Conv2D)            │ (None, 32, 32, 256)       │         524,544 │ leaky_re_lu_48[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_83        │ (None, 32, 32, 256)       │           1,024 │ conv2d_56[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_49 (LeakyReLU)    │ (None, 32, 32, 256)       │               0 │ batch_normalization_83[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_57 (Conv2D)            │ (None, 16, 16, 512)       │       2,097,664 │ leaky_re_lu_49[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_84        │ (None, 16, 16, 512)       │           2,048 │ conv2d_57[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_50 (LeakyReLU)    │ (None, 16, 16, 512)       │               0 │ batch_normalization_84[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_58 (Conv2D)            │ (None, 8, 8, 512)         │       4,194,816 │ leaky_re_lu_50[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_85        │ (None, 8, 8, 512)         │           2,048 │ conv2d_58[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_51 (LeakyReLU)    │ (None, 8, 8, 512)         │               0 │ batch_normalization_85[0]… │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 54,429,315 (207.63 MB)

 Trainable params: 54,419,459 (207.59 MB)

 Non-trainable params: 9,856 (38.50 KB)

None
